In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut


Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
dimension = (64,64)
batch_size = 100
channels = 5
optimizer = Adam( lr = 1e-3 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
#TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)


Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [11]:
def multinulli(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):

    
    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8

    conv03_pool = Dense(64*64)(conv03_pool)
    conv03_pool = Dense(64*64)(conv03_pool)
    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64
    output = Conv2D(7, (64, 64),padding="same",ac)(up01)  # 1 x 64x64

    #output = Flatten()(output)
    #output = Dense(4096)(output)
    output = tfp.layers.OneHotCategorical(event_size=(64,64,7))(output)
    model = Model(inputs=inputs, outputs=output)
    return model

In [12]:
print(len(test[0][0]),len(train[0][0]))
print(test[0][1].shape)

100 100
(100, 64, 64, 7)


In [13]:
t = Trainer(multinulli,
                    NLL,
                    (train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

print("len train,val",len(train),len(test))

[Load file failed]  ./model_data/multinulli_function/multinulli_function64x64x5.h5
[Load file failed]  ./model_data/multinulli_function/multinulli_function64x64x5history.json
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 64, 64, 10)   460         input_3[0][0]                    
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 64, 64, 10)   0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_8 

In [6]:
model = t.model
print(model(train[0][0]).shape)
print(train[0][1].shape)

(100, 64, 64, 7)
(100, 64, 64, 7)


In [14]:
t.fit(20)

Epoch 1/20
 17/231 [=>............................] - ETA: 47s - loss: 24.2279 - mse: 0.1426 - mae: 0.1432

KeyboardInterrupt: 

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
history = t.history
def plotHistory():
    # Plot training & validation accuracy values
    plt.plot(history['mse'])
    plt.plot(history['val_mse'])
    plt.title('Model MSE')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plotHistory()

In [ ]:
model = t.model


In [ ]:
x,y = test[0]
print(y.shape)
pred = model(x)
print(pred.mean()[-1])
        

In [ ]:
class fromCategorical(object):
    def __init__(self, conditions):
        super(fromCategorical, self).__init__()
        self.conditions = np.array(conditions)
        self.numClasses = len(self.conditions) -1
        
    def __call__(self,array):
        img = np.zeros( (array.shape[0],array.shape[1],1) )
        x,y = array.shape[:2]
        
        def mapTo(l):
            if np.argmax(l) != 0:
                img[i,j] = self.conditions[np.argmax(l)]
        for i in range(x):
            for j in range(y):
                mapTo(array[i,j])
        return img
        #print(np.argmax(array,axis=-1))
        #print(np.argmax(np.array(array),axis=-1).max())
        

In [ ]:
fromcat = fromCategorical(categorical_list)
img = []
for i in range(pred.shape[0]):
    img.append(fromcat(pred[i,:,:,:]))

label = []
for i in range(y.shape[0]):
    label.append(fromcat(y[i,:,:,:]))

In [ ]:
print(label[0].shape)

In [ ]:
fig, axes = plt.subplots(20,2, figsize=(8,8*20),dpi=64)

for batch,img in enumerate(pred[:20]):
    axes[batch,0].imshow(img[:,:,0],cmap="gray")
    axes[batch,1].imshow(label[batch][:,:,0],cmap="gray")